In [6]:
import pandas as pd
import yfinance as yf
import configparser
import sqlalchemy

In [7]:
config = configparser.ConfigParser()
config.read("config.ini")
user = config["local_postgres"]["DB_USER"]
pw = config["local_postgres"]["DB_PW"]
address = config["local_postgres"]["DB_ADDRESS"]
port = config["local_postgres"]["DB_PORT"]
name = config["local_postgres"]["DB_NAME"]

In [8]:
con_str = f"postgresql://{user}:{pw}@{address}:{port}/{name}"
con = sqlalchemy.create_engine(con_str)

In [9]:
query = """insert into finance.symbols values
     ('MSFT', 'Microsoft', 'United States')"""
con.execute(query, con)

In [14]:
def insert_symbol(con, ticker):
    # Daten aus Ticker auslesen
    symbol = ticker.info["symbol"]
    name = ticker.info["longName"]
    country = ticker.info["country"]
    # Prüfen, ob Symbol schon vorhanden
    query = f"""select count(1) from finance.symbols 
                where symbol = '{symbol}';"""
    df = pd.read_sql(query,con)
    vorhanden = df.iloc[0,0] == 1
    # Daten in Datenbank schreiben
    if not vorhanden:
        query = f"""insert into finance.symbols values
        ('{symbol}', '{name}', '{country}');"""    
        con.execute(query)
    else:
        print("Eintrag existiert schon")

In [13]:
symbol = 'MSFT'
query = f"""select count(1) from finance.symbols 
    where symbol = '{symbol}'"""
df = pd.read_sql(query, con)
con.execute(query)

In [15]:
t = yf.Ticker("AAPL")
insert_symbol(con, t)

Eintrag existiert schon
